# ทดสอบการค้นคืนข่าวที่น่าชื่อถือจากทุกแหล่ง (cofact/anti/sure) ด้วย cosine similarity + TF-IDF (ใช้เฉพาะหัวข้อข่าว(header จาก cofact_refer + anti_info + sure_info.json) 10/12/2021)

In [106]:
import numpy as np
import nltk
import re
import pandas as pd 
import json
import os
import pickle
import csv

from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from string import punctuation

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

nltk.download('words')
th_stop = thai_stopwords()
en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

def split_word(text):
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และอักขระพิเศษ
    tokens = [i for i in tokens if (not i in th_stop) & (not i in en_stop) & (not i in punctuation) & (not i in ["'",'"','“','”','‘','’','\n',"None", ' '])] 
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## ดึงข้อมูลของ cofact

In [107]:
def read_cofact_refer():
    # Opening CSV file
    root_path = os.getcwd()
    path = os.path.join(root_path, '../Scrapy/Scrapy_project/spiders/results/cofact/cofact_refer.csv')
    f = open(path, encoding="utf8")
    
    # returns CSV object as
    # a dictionary
    csvreader = csv.reader(f)

    # read header from CSV file
    header = []
    header = next(csvreader)
    
    rows = []
    for row in csvreader:
        tmp = []
        tmp.append(row[1])
        tmp.append(row[3])
        rows.append(tmp)

    # Closing file
    f.close()
    
    return rows

In [108]:
cofact_refer_text_list = read_cofact_refer()

In [109]:
len(cofact_refer_text_list)

2317

-----------------------------------------------
## ดึงข้อมูลของ anti

In [110]:
def read_anti_refer():
    # Opening CSV file
    root_path = os.getcwd()
    path = os.path.join(root_path, '../Scrapy/Scrapy_project/spiders/results/anti/anti_info.csv')
    f = open(path, encoding="utf8")
    
    # returns CSV object as
    # a dictionary
    csvreader = csv.reader(f)
    
    # read header from CSV file
    header = []
    header = next(csvreader)

    rows = []
    for row in csvreader:
        tmp = []
        tmp.append(row[1])
        tmp.append(row[3])
        rows.append(tmp)

    # Closing file
    f.close()
    
    return rows

In [111]:
anti_refer_text_list = read_anti_refer()

In [112]:
anti_refer_text_list

[['ข่าวปลอม “บิ๊กตู่” เรียกเก็บเงินภาษี 7 % “ก้าวคนละก้าว”',
  'https://www.antifakenewscenter.com/%e0%b8%82%e0%b9%88%e0%b8%b2%e0%b8%a7%e0%b8%9b%e0%b8%a5%e0%b8%ad%e0%b8%a1-%e0%b8%81%e0%b9%89%e0%b8%b2%e0%b8%a7%e0%b8%84%e0%b8%99%e0%b8%a5%e0%b8%b0%e0%b8%81%e0%b9%89%e0%b8%b2%e0%b8%a7/'],
 ['เว็บคลิกเบทผุดข่าวปลอม เสี่ยตัน ไม่ช่วย พี่ตูน ชาวเน็ตด่าขรมโปรดอย่าแชร์',
  'https://www.antifakenewscenter.com/%e0%b9%80%e0%b8%a7%e0%b9%87%e0%b8%9a%e0%b8%84%e0%b8%a5%e0%b8%b4%e0%b8%81%e0%b9%80%e0%b8%9a%e0%b8%97%e0%b8%9c%e0%b8%b8%e0%b8%94%e0%b8%82%e0%b9%88%e0%b8%b2%e0%b8%a7%e0%b8%9b%e0%b8%a5%e0%b8%ad%e0%b8%a1/'],
 ['สธ.เตือน “อย่าเชื่อ-อย่าแชร์” ข่าวปลอมอ้างมีคนตายจากฝุ่นPM2.5',
  'https://www.antifakenewscenter.com/%e0%b8%82%e0%b9%88%e0%b8%b2%e0%b8%a7%e0%b8%9b%e0%b8%a5%e0%b8%ad%e0%b8%a1-%e0%b8%84%e0%b8%99%e0%b8%95%e0%b8%b2%e0%b8%a2%e0%b8%88%e0%b8%b2%e0%b8%81%e0%b8%9d%e0%b8%b8%e0%b9%88%e0%b8%99pm2-5/'],
 ['“ไอซ์ ปรีชญา” เดือด เว็บปลอมกุข่าว โดนรวบปาร์ตี้ยา เตรียมแจ้งความกลับ',
  'https://www.antifakene

In [113]:
len(anti_refer_text_list)

2302

-----------------------------------------------
## ดึงข้อมูลของ sure

In [114]:
def read_sure_refer():
    # Opening JSON file
    root_path = os.getcwd()
    path = os.path.join(root_path, '../Scrapy/Scrapy_project/spiders//fetch file/sure_info.json')
    f = open(path, encoding="utf8")
    
    # returns JSON object as
    # a dictionary
    sure_refer_text_data = json.load(f)

    # Closing file
    f.close()
    return sure_refer_text_data

In [115]:
sure_refer_text_data = read_sure_refer()

In [116]:
len(sure_refer_text_data)

2068

In [117]:
def get_sure_refer_info(sure_refer_text_data): #สำหรับดึงหัวข้อข่าวอ้างอิง (header) และลิงของข่าวอ้างอิง (link)
    refer_text_list = []

    for refer_text in range(len(sure_refer_text_data)):
        tmp = []
        tmp.append(sure_refer_text_data[refer_text]['header'])
        tmp.append(sure_refer_text_data[refer_text]['link'])
        refer_text_list.append(tmp)

    return refer_text_list

In [118]:
sure_refer_text_list = get_sure_refer_info(sure_refer_text_data)

In [119]:
sure_refer_text_list

[['ชัวร์ก่อนแชร์: ตาลีบันห้ามฉีดวัคซีนโควิด 19 ในอัฟกานิสถาน จริงหรือ?',
  'https://tna.mcot.net/latest-news-825069'],
 ['ชัวร์ก่อนแชร์: WHO ประกาศให้โควิด 19 เป็นโรคประจำถิ่น จริงหรือ?',
  'https://tna.mcot.net/latest-news-824200'],
 ['ชัวร์ก่อนแชร์: WHO ไม่ยอมรับภูมิคุ้มกันธรรมชาติ จริงหรือ?',
  'https://tna.mcot.net/latest-news-823234'],
 ['ชัวร์ก่อนแชร์: สหรัฐใช้ยา Ivermectin รักษาผู้อพยพที่ติดเชื้อโควิด จริงหรือ?',
  'https://tna.mcot.net/sureandshare-822486'],
 ['ชัวร์ก่อนแชร์: ญี่ปุ่นยกเลิกวัคซีน Moderna ใช้ Ivermectin รักษาโควิด จริงหรือ?',
  'https://tna.mcot.net/latest-news-821794'],
 ['ชัวร์ก่อนแชร์: Oxford รับรองการรักษาโควิด 19 ด้วย Ivermectin จริงหรือ?',
  'https://tna.mcot.net/latest-news-821239'],
 ['ชัวร์ก่อนแชร์: หัวหน้านักวิทย์ WHO โดนดำเนินคดีกรณีด้อยค่า Ivermectin จริงหรือ?',
  'https://tna.mcot.net/latest-news-820342'],
 ['ชัวร์ก่อนแชร์: สวิตเซอร์แลนด์จำหน่ายยากำจัดวัคซีนโควิด 19 จากร่างกาย จริงหรือ?',
  'https://tna.mcot.net/latest-news-819351'],
 ['ชัวร์ก่อนแชร์

In [120]:
len(sure_refer_text_list)

2068

## รวมทุก list จากทุกแหล่งข่าวเข้าด้วยกัน

In [121]:
cofact_refer_text_list[:2]

[['น้ำชาไม่สามารถรักษาโรคแพ้อาหารได้ โปรดอย่าหลงเชื่อ',
  'https://sure.oryor.com/index.php/detail/media_specify/605'],
 ['มะนาวเกลือแก้ไมเกรน มั่วมาก อย่าไปเชื่อ',
  'https://sure.oryor.com/index.php/detail/media_specify/426']]

In [122]:
anti_refer_text_list[:2]

[['ข่าวปลอม “บิ๊กตู่” เรียกเก็บเงินภาษี 7 % “ก้าวคนละก้าว”',
  'https://www.antifakenewscenter.com/%e0%b8%82%e0%b9%88%e0%b8%b2%e0%b8%a7%e0%b8%9b%e0%b8%a5%e0%b8%ad%e0%b8%a1-%e0%b8%81%e0%b9%89%e0%b8%b2%e0%b8%a7%e0%b8%84%e0%b8%99%e0%b8%a5%e0%b8%b0%e0%b8%81%e0%b9%89%e0%b8%b2%e0%b8%a7/'],
 ['เว็บคลิกเบทผุดข่าวปลอม เสี่ยตัน ไม่ช่วย พี่ตูน ชาวเน็ตด่าขรมโปรดอย่าแชร์',
  'https://www.antifakenewscenter.com/%e0%b9%80%e0%b8%a7%e0%b9%87%e0%b8%9a%e0%b8%84%e0%b8%a5%e0%b8%b4%e0%b8%81%e0%b9%80%e0%b8%9a%e0%b8%97%e0%b8%9c%e0%b8%b8%e0%b8%94%e0%b8%82%e0%b9%88%e0%b8%b2%e0%b8%a7%e0%b8%9b%e0%b8%a5%e0%b8%ad%e0%b8%a1/']]

In [123]:
sure_refer_text_list[:2]

[['ชัวร์ก่อนแชร์: ตาลีบันห้ามฉีดวัคซีนโควิด 19 ในอัฟกานิสถาน จริงหรือ?',
  'https://tna.mcot.net/latest-news-825069'],
 ['ชัวร์ก่อนแชร์: WHO ประกาศให้โควิด 19 เป็นโรคประจำถิ่น จริงหรือ?',
  'https://tna.mcot.net/latest-news-824200']]

In [124]:
all_refer_text_list = []
all_refer_text_list = cofact_refer_text_list + anti_refer_text_list + sure_refer_text_list

In [125]:
all_refer_text_list

[['น้ำชาไม่สามารถรักษาโรคแพ้อาหารได้ โปรดอย่าหลงเชื่อ',
  'https://sure.oryor.com/index.php/detail/media_specify/605'],
 ['มะนาวเกลือแก้ไมเกรน มั่วมาก อย่าไปเชื่อ',
  'https://sure.oryor.com/index.php/detail/media_specify/426'],
 ['อย่าหลงเชื่อ “นาโน สเปรย์” อ้างฆ่าเชื้อไวรัสและแบคทีเรีย',
  'https://oryor.com/%E0%B8%AD%E0%B8%A2/detail/media_news/1801?fbclid=IwAR3ehr8qOSiduYsLaWphsDOwgcmuR0nHtf3O1m_9IJmoslmJt-Oky8TFzNI'],
 ['ดื่มกาแฟวันละสามแก้วทำให้หน้าอกเล็กลงไม่จริง ',
  'https://sure.oryor.com/index.php/detail/media_specify/701'],
 ['คิ้วดกดำด้วยการทาปิโตรเลียม เจลลี่ ไม่จริง',
  'https://sure.oryor.com/index.php/detail/media_specify/702'],
 ['อย่าหลงเชื่อ! ‘อ.เจษฎ์’ ยัน ‘ไวรัสโคโรนา’ ถูกตัดต่อพันธุกรรมผสม HIV เป็นอาวุธชีวภาพ ไม่เป็นความจริง',
  'https://www.matichon.co.th/news-monitor/news_1939718'],
 ['ข่าวปลอม อย่าแชร์! ยาพาราเซตามอลมีส่วนผสมของไวรัสแมคชูโป',
  'https://www.antifakenewscenter.com/%E0%B8%82%E0%B9%88%E0%B8%B2%E0%B8%A7%E0%B8%9B%E0%B8%A5%E0%B8%AD%E0%B8%A1-%E0%B8%AD%

In [126]:
len(all_refer_text_list)

6687

## ------------------------------------------------------------------------------
## สร้าง dataframe ของ cofact สำหรับอ้างถึงตอนแนบลิงค์ url แสดงผล

In [127]:
all_refer_text = []
all_refer_url = []
for i in range(len(all_refer_text_list)):
    all_refer_text.append(all_refer_text_list[i][0]) #list ของส่วนเนื้อหาเท่านั้น
    all_refer_url.append(all_refer_text_list[i][1]) #list ของ url เท่านั้น


In [128]:
#ทำ list ให้เป็น dataframe
all_original_text_and_headline_news_df = pd.DataFrame(list(zip(all_refer_text, all_refer_url)), columns=["หัวข้อข่าวอ้างอิงของ ทุกแหล่งข่าว", "URL ของ ทุกแหล่งข่าว"])

all_original_text_and_headline_news_df

,หัวข้อข่าวอ้างอิงของ ทุกแหล่งข่าว,URL ของ ทุกแหล่งข่าว
0,น้ำชาไม่สามารถรักษาโรคแพ้อาหารได้ โปรดอย่าหลงเ...,https://sure.oryor.com/index.php/detail/media_...
1,มะนาวเกลือแก้ไมเกรน มั่วมาก อย่าไปเชื่อ,https://sure.oryor.com/index.php/detail/media_...
2,อย่าหลงเชื่อ “นาโน สเปรย์” อ้างฆ่าเชื้อไวรัสแล...,https://oryor.com/%E0%B8%AD%E0%B8%A2/detail/me...
3,ดื่มกาแฟวันละสามแก้วทำให้หน้าอกเล็กลงไม่จริง,https://sure.oryor.com/index.php/detail/media_...
4,คิ้วดกดำด้วยการทาปิโตรเลียม เจลลี่ ไม่จริง,https://sure.oryor.com/index.php/detail/media_...
...,...,...
6682,ชัวร์ก่อนแชร์ : ระวัง 10 ขนมขบเคี้ยวอันตราย! จ...,https://tna.mcot.net/tna-93789
6683,ชัวร์ก่อนแชร์ : เปิดแอร์ใส่หน้าเด็ก ทำให้เป็นอ...,https://tna.mcot.net/tna-93790
6684,ชัวร์ก่อนแชร์ : มะเร็งต้องเลิกบำรุงจริงหรือ?,https://tna.mcot.net/tna-93791
6685,ชัวร์ก่อนแชร์ : ยาแอสไพรินช่วยซักผ้าขาวสะอาด จ...,https://tna.mcot.net/tna-100770


In [129]:
all_headline_tokens_list = [split_word(txt) for txt in all_refer_text] #นำ list ของเนื้อหามาตัดคำ

In [130]:
all_tokens_list_j = [','.join(tkn) for tkn in all_headline_tokens_list]

----------------------------------------------------------------------------------
# สร้าง vocabulary set

In [131]:
print(all_tokens_list_j[0].split(",")[0:2])


['น้ำชา', 'รักษาโรค']


In [132]:
## Create Vocabulary
tokens_list = []

for words in all_tokens_list_j:
    # print(words)
    temp_list = words.split(",")
    # print(temp_list)
    for i in temp_list:
        tokens_list.append(i)

vocabulary = set(tokens_list)
vocabulary = list(vocabulary)

In [133]:
len(vocabulary)

7062

--------------------------------------------------------------------------------------

In [134]:
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
original_c_feat = tvec.fit_transform(all_tokens_list_j)

### Save Trained TFID Model

In [135]:
### Save model
root_path = os.getcwd()
path = os.path.join(root_path, 'all-tfid.pkl')
print(path)
with open(path,'wb') as handle:
    pickle.dump(original_c_feat, handle)

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\all-tfid.pkl


In [136]:
### load model
root_path = os.getcwd()
path = os.path.join(root_path, 'all-tfid.pkl')
print(path)
t = pickle.load(open(path,'rb'))

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\all-tfid.pkl


### Save Vocabulary

In [137]:
### Save Vacabulary
root_path = os.getcwd()
path = os.path.join(root_path, 'vocabulary_all.txt')
print(path)
with open(path, "w", encoding="utf-8") as file:
    file.write(str(vocabulary))

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\vocabulary_all.txt


In [138]:
### load Vacabulary
root_path = os.getcwd()
path = os.path.join(root_path, 'vocabulary_all.txt')
print(path)
with open(path, "r", encoding="utf-8") as file:
    data2 = eval(file.readline())

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\vocabulary_all.txt


In [139]:
data2

['',
 'เสมหะ',
 'ตรัง',
 'เทศบาล',
 'IMF',
 'สูตร',
 'ทันใจ',
 'โป๊ะแตก',
 'แทบ',
 'ขนาดเล็ก',
 'อัฟกานิสถาน',
 'ดินแดง',
 'ใบแดง',
 'ประกาศ',
 'นายแพทย์',
 'หมูหยอง',
 'ชิด',
 'ลั่น',
 'อู่',
 'สกลนคร',
 'อุตสาหการ',
 'ล้างมือ',
 'ปู',
 'แม้ว',
 'List',
 'เก๊าท์',
 'เยี่ยมชม',
 'ผัด',
 'ขมับ',
 'ปรากฎการณ์',
 'อั้น',
 'เฉียบพลัน',
 'ความดันโลหิต',
 'ร้ายแรง',
 'ที่จะ',
 'ฝอย',
 'Spray',
 'อัตราดอกเบี้ย',
 'อำเภอ',
 'พลัส',
 'แห้ง',
 'การสั่น',
 'ตรวจจับ',
 'ร.ง.',
 'มีน',
 'อันตราย',
 'โรคหวัด',
 'นิด้า',
 'บาดเจ็บ',
 'Alternative',
 'ไล่ออก',
 'อาหารทะเล',
 'น้อม',
 '“ธา',
 'อนุภาค',
 'ต่อไป',
 'หน้าท้อง',
 'กะพริบ',
 'ขสมก.',
 'ปิดเทอม',
 'Moisturizer',
 'อาจารย์',
 'ปชช.',
 'ทุ่งสง',
 'เข้าพรรษา',
 'ตัวตืด',
 'กระ',
 'วันหยุดราชการ',
 'ธร',
 'คู่มือ',
 'ขายส่ง',
 'หมดสภาพ',
 'สนามมวย',
 'ชายหาด',
 'เป็นได้',
 'แปะ',
 'วดี',
 'สัตหีบ',
 'การผลิต',
 'ทัพเรือ',
 'รักษาโรค',
 'การไฟฟ้าส่วนภูมิภาค',
 'องค์การอนามัยโลก',
 'โกโก้',
 'สัน',
 'ญี่ปุ่น',
 'ให้อาหาร',
 'ลูกเกด',
 'ปน',
 'คลิป

In [140]:
len(data2)

7062

## Create vector for Query/search keywords

In [141]:
## Create vector for Query/search keywords
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))
    
    x = tvec.transform(tokens)
    
    x = x.A[0]
    return x

### Calculate Cosine similarity of trained Tfidf to input query


In [142]:
norm_original_c_feat = normalize(original_c_feat)
norm_original_c_feat = norm_original_c_feat.toarray()
norm_original_c_feat = norm_original_c_feat.T

In [143]:
query_vector = []
def cosine_similarity_T(k, query):
    tokens = split_word(str(query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =str(tokens)

    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    
    d_cosines = []
    
    query_vector = gen_vector_T(q_df['q_clean'])
    query_vector = query_vector.reshape((1,-1))

    d_cosines = np.dot(normalize(query_vector),norm_original_c_feat)
    
    list_d_cosines = d_cosines[0].tolist()
    
    out = np.array(list_d_cosines).argsort()[-k:][::-1]

    list_d_cosines.sort()
    a = pd.DataFrame()
    for i in out:
        a.loc[i,'index'] = str(i)
        a.loc[i,'Subject'] = all_original_text_and_headline_news_df['หัวข้อข่าวอ้างอิงของ ทุกแหล่งข่าว'][i]

    list_d_cosines.sort(reverse=True)
    
    for j in range(k):
        a.loc[out[j],'Score'] = list_d_cosines[j]
        
    return a

In [144]:
%time all_result = cosine_similarity_T(10,'กาแฟ')

Wall time: 72 ms


In [145]:
all_result

,index,Subject,Score
805,805,กาแฟ ต้านโควิดได้ จริงหรือไม่,0.736566
6491,6491,ชัวร์ก่อนแชร์ : 20 ประโยชน์ของกาแฟ จริงหรือ?,0.691494
6179,6179,ชัวร์ก่อนแชร์ : ดื่มกาแฟอย่างไรไม่ให้เสียให้สุ...,0.630046
5615,5615,ชัวร์ก่อนแชร์ : 5 โรคห้ามดื่มกาแฟ จริงหรือ?,0.612935
24,24,แชร์ผิดผิด กาแฟใส่เนย,0.545241
945,945,ดื่มชาปลอดภัยกว่ากาแฟจริงหรือ,0.532678
1461,1461,กาแฟกับภาวะกระดูกพรุน,0.511173
6557,6557,ชัวร์ก่อนแชร์ : ชามีกาเฟอีนมากกว่ากาแฟจริงหรือ?,0.500034
5901,5901,ชัวร์ก่อนแชร์ : คลิปเตือนกาแฟผสมดินประสิว จริง...,0.458314
2891,2891,ข่าวปลอม อย่าแชร์! กาแฟสารสกัดจากธรรมชาติ ต้าน...,0.444004


In [146]:
all_result_with_url = pd.DataFrame()
for i in range(len(all_result)):
    all_result_with_url.loc[i,'index'] = all_result.iloc[i]["index"]
    all_result_with_url.loc[i,'text'] = all_result.iloc[i]["Subject"]
    all_result_with_url.loc[i,'url'] = all_original_text_and_headline_news_df["URL ของ ทุกแหล่งข่าว"][int(all_result.iloc[i]["index"])]

In [147]:
all_result_with_url

,index,text,url
0,805,กาแฟ ต้านโควิดได้ จริงหรือไม่,https://sure.oryor.com/index.php/detail/media_...
1,6491,ชัวร์ก่อนแชร์ : 20 ประโยชน์ของกาแฟ จริงหรือ?,https://tna.mcot.net/tna-75433
2,6179,ชัวร์ก่อนแชร์ : ดื่มกาแฟอย่างไรไม่ให้เสียให้สุ...,https://tna.mcot.net/tna-197777
3,5615,ชัวร์ก่อนแชร์ : 5 โรคห้ามดื่มกาแฟ จริงหรือ?,https://tna.mcot.net/tna-370528
4,24,แชร์ผิดผิด กาแฟใส่เนย,https://oryor.com/%E0%B8%AD%E0%B8%A2/detail/me...
5,945,ดื่มชาปลอดภัยกว่ากาแฟจริงหรือ,https://sure.oryor.com/index.php/detail/media_...
6,1461,กาแฟกับภาวะกระดูกพรุน,https://oryor.com/%E0%B8%AD%E0%B8%A2/detail/me...
7,6557,ชัวร์ก่อนแชร์ : ชามีกาเฟอีนมากกว่ากาแฟจริงหรือ?,https://tna.mcot.net/tna-9776
8,5901,ชัวร์ก่อนแชร์ : คลิปเตือนกาแฟผสมดินประสิว จริง...,https://tna.mcot.net/tna-278578
9,2891,ข่าวปลอม อย่าแชร์! กาแฟสารสกัดจากธรรมชาติ ต้าน...,https://www.antifakenewscenter.com/%e0%b8%82%e...


---------------------------------------